| Variable | Definition                                 | Key                                                                                                                                                                                                |
|----------|--------------------------------------------|----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| survival | Survival                                   | 0 = No, 1 = Yes                                                                                                                                                                                    |
| pclass   | Ticket class                               | 1 = 1st, 2 = 2nd, 3 = 3rd  pclass : A proxy for socio-economic status (SES)  1st = Upper  2nd = Middle  3rd = Lower                                                                                |
| sex      | Sex                                        |                                                                                                                                                                                                    |
| Age      | Age in years                               | Age is fractional if less than 1. If the age is estimated, is it in the form of xx.5                                                                                                               |
| sibsp    | # of siblings / spouses aboard the Titanic | The dataset defines family relations in this way...  Sibling = brother, sister, stepbrother, stepsister  Spouse = husband, wife (mistresses and fiancés were ignored)                              |
| parch    | # of parents / children aboard the Titanic | The dataset defines family relations in this way...  Parent = mother, father  Child = daughter, son, stepdaughter, stepson  Some children travelled only with a nanny, therefore parch=0 for them. |
| ticket   | Ticket number                              |                                                                                                                                                                                                    |
| fare     | Passenger fare                             |                                                                                                                                                                                                    |
| cabin    | Cabin number                               |                                                                                                                                                                                                    |
| embarked | Port of Embarkation                        | C = Cherbourg, Q = Queenstown, S = Southampton                                                                                                                                                     |

In [81]:
import pandas as pd
import numpy as np

In [2]:
df_test = pd.read_csv('./data/test.csv')
df_train = pd.read_csv('./data/train.csv')

In [3]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null float64
Cabin          91 non-null object
Embarked       418 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [4]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [6]:
df_test.Age.mean()

30.272590361445783

In [8]:
df_train.Age.mean()

29.69911764705882

In [18]:
df_train[df_train.Pclass==3].Age.mean()

25.14061971830986

In [20]:
df_train[df_train.Pclass==2].Age.mean()

29.87763005780347

In [21]:
df_train[df_train.Pclass==1].Age.mean()

38.233440860215055

I want to extract the titles of the passangers on the ship. To do this I ceeated a simple regex function and filled it with the titles I an aware of. Unfortunately titles that im not awware of appear as an empty list. In python empty lists are concidered false for boolean statements so I used an np.where, which is a simple if then statemtent that acts on the entirity ot the pandas column, to check if the valus in the column is true and if it is not to replace the empty list with a NAN. Then I sorted the list by NaN values in the title column to find the missing titles.

In [73]:
import re
str_srch = r'(?:Mr\.|Mrs\.|Miss\.|Master\.|Don\.|Rev\.|Dr\.)+'

In [97]:
df_train['title'] = [re.findall(str_srch,i) for i in df_train.Name.values]

In [129]:
df_train.title = np.where(df_train.title.values, df_train.title.values, np.NaN)

In [131]:
df_train[df_train.title.isna()]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,title
369,370,1,1,"Aubart, Mme. Leontine Pauline",female,24.0,0,0,PC 17477,69.3000,B35,C,NaN
443,444,1,2,"Reynaldo, Ms. Encarnacion",female,28.0,0,0,230434,13.0000,NaN,S,NaN
449,450,1,1,"Peuchen, Major. Arthur Godfrey",male,52.0,0,0,113786,30.5000,C104,S,NaN
536,537,0,1,"Butt, Major. Archibald Willingham",male,45.0,0,0,113050,26.5500,B38,S,NaN
556,557,1,1,"Duff Gordon, Lady. (Lucille Christiana Sutherl...",female,48.0,1,0,11755,39.6000,A16,C,NaN
599,600,1,1,"Duff Gordon, Sir. Cosmo Edmund (""Mr Morgan"")",male,49.0,1,0,PC 17485,56.9292,A20,C,NaN
641,642,1,1,"Sagesser, Mlle. Emma",female,24.0,0,0,PC 17477,69.3000,B35,C,NaN
647,648,1,1,"Simonius-Blumer, Col. Oberst Alfons",male,56.0,0,0,13213,35.5000,A26,C,NaN
694,695,0,1,"Weir, Col. John",male,60.0,0,0,113800,26.5500,NaN,S,NaN
710,711,1,1,"Mayne, Mlle. Berthe Antonine (""Mrs de Villiers"")",female,24.0,0,0,PC 17482,49.5042,C90,C,NaN


In [4]:
str_srch = r'(?:Mr\.|Mrs\.|Miss\.|Master\.|Don\.|Rev\.|Dr\.|Jonkheer\.|Countess\.|Capt\.|Mlle\.|Col\.|Sir\.|Lady\.|Major\.|Ms\.|Mme\.|)+'